#INIT

In [0]:
from pyspark.sql.functions import col, lit, concat

COVERED_DATE = {
    "start": "2025-01-01",
    "end": "2025-01-31"
}

#Read From Silver Loans And Join In Bronze Loandiscounts

In [0]:
query = f"""
        SELECT
            bld.Id AS id,
            bld.LoanId AS loanid,
            bld.BranchCode AS branchcode,
            bls.Name AS loanschemename,
            sl.status AS status,
            bld.LoanSchemeDiscountId AS loanschemediscountid,
            bldt.Code AS loandiscounttypecode,
            bldt.Name AS loandiscountname,
            bld.ReferenceCode AS referencecode,
            bld.Remarks AS remarks,
            bldt.AccountCode AS accountcode,
            bldt.LoanTranxType AS loantranxtype,
            sl.loandate AS loandate,
            bld.Amount AS amount,
            bld.Deleted AS deleted,
            bld.LastModified AS lastmodified
        FROM workspace.silver.silver_loans sl
        LEFT JOIN workspace.bronze.bronze_loandiscounts bld
            ON sl.id = bld.LoanId
        LEFT JOIN workspace.bronze.bronze_loanschemediscounts blsd
            ON bld.LoanSchemeDiscountId = blsd.Id
        LEFT JOIN workspace.bronze.bronze_loandiscounttypes bldt
            ON blsd.LoanDiscountTypeId = bldt.Id
        LEFT JOIN workspace.bronze.bronze_loanschemes bls
            ON blsd.LoanSchemeId = bls.Id
        WHERE sl.transactiondate >= DATE('2025-01-01')
        AND sl.transactiondate < DATE('2025-01-31') + INTERVAL 1 DAY
"""

loandiscounts_df = spark.sql(query)

#Data Transformations

##Assign Charge Group Base on Loantanxtype and Account Code

In [0]:
loandiscounts_df = loandiscounts_df.withColumn("discountgrp", concat(lit("D0"), col("loantranxtype"), lit("_"), col("accountcode")))

#Write to Silver Loandiscounts


In [0]:
loandiscounts_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("silver.silver_loandiscounts")